In [174]:
import pandas as pd;
import numpy as np;
from glob import glob;
import time;
import datetime;
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
import pickle

In [87]:
print(datetime.datetime.now().time())
for fil in glob('part-0000*.csv'):    
    perFileDf = pd.DataFrame()
    start_time = time.time()
    print ("Started processing the file : "+ fil)
    for df in pd.read_csv(fil,iterator=True,chunksize=10000000):                
                
        df["speedFactor"] = (np.sqrt((df['acceleration_x'])*(df['acceleration_x']) + (df['acceleration_y'])*(df['acceleration_y']) + (df['acceleration_z'])*(df['acceleration_z'])))/(10 + df["Speed"])        
        
        df["absAccX"] = np.abs(df['acceleration_x'])
        df["absAccY"] = np.abs(df['acceleration_y'])
        df["absAccZ"] = np.abs(df['acceleration_z'])

        df["absGyroX"] = np.abs(df['gyro_x'])
        df["absGyroY"] = np.abs(df['gyro_y'])
        df["absGyroZ"] = np.abs(df['gyro_z'])
                                
        #print (df["speedFactor"].describe(include='all'))
        #print ("==========")
        newDf=df.groupby('bookingID').agg({'absAccX':'mean', 'absAccY':'mean','absAccZ':'mean','absGyroX':'mean', 'absGyroY':'mean','absGyroZ':'mean','speedFactor':'mean'})
        newDf = newDf.reset_index()
        #print (newDf.head())
        #print ("***************")
        perFileDf =  perFileDf.append(newDf)       
    
    print ("Done with file : "+ fil)   
    elapsed_time = time.time() - start_time
    print ("Time to process file : " + str(fil)  + " is " + str(elapsed_time))
    start_time = time.time()
    perFileDf.to_csv("df_"+fil)
    elapsed_time = time.time() - start_time
    print ("Time to write Df file : " + "df_"+str(fil)  + " is " + str(elapsed_time))

print(datetime.datetime.now().time())
print("End of phase-1")

18:55:12.105282
Started processing the file : part-00009-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv
Done with file : part-00009-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv
Time to process file : part-00009-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv is 14.27955150604248
Time to write Df file : df_part-00009-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv is 1.20078706741333
18:55:27.591669
End of the program


In [97]:
print(datetime.datetime.now().time())
consolidatedDf = pd.DataFrame()
for fil in glob('df_part-0000*.csv'):    
    start_time = time.time()
    print ("Started processing the file : "+ fil)
    for df in pd.read_csv(fil,iterator=True,chunksize=10000000):                
        print ("Current Df shape" + str(df.shape))
        consolidatedDf = consolidatedDf.append(df)               
        print ("Consolidated Df shape" + str(consolidatedDf.shape))
    print ("Done processing the file : "+ fil)    
    elapsed_time = time.time() - start_time
    print ("Time to process file : " + str(fil)  + " is " + str(elapsed_time))    

consolidatedDf.to_csv("consolidatedDf.csv")
grpConsolidatedDf =consolidatedDf.groupby('bookingID').agg({'absAccX':'mean', 'absAccY':'mean','absAccZ':'mean','absGyroX':'mean', 'absGyroY':'mean','absGyroZ':'mean','speedFactor':'mean'})
print ("After Consolidated Df shape" + str(grpConsolidatedDf.shape))
grpConsolidatedDf.to_csv("grpConsolidatedDf.csv")
print(datetime.datetime.now().time())

print("End of phase-2")

19:14:36.086200
Started processing the file : df_part-00000-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv
Current Df shape(20000, 9)
Consolidated Df shape(20000, 9)
Done processing the file : df_part-00000-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv
Time to process file : df_part-00000-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv is 0.19759488105773926
Started processing the file : df_part-00001-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv
Current Df shape(20000, 9)
Consolidated Df shape(40000, 9)
Done processing the file : df_part-00001-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv
Time to process file : df_part-00001-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv is 0.1946403980255127
Started processing the file : df_part-00002-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv
Current Df shape(20000, 9)
Consolidated Df shape(60000, 9)
Done processing the file : df_part-00002-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv
Time to process file : df_part-00002-e6120af0-10c2-4248-97c4-81ba

In [98]:
grpDf =  pd.read_csv("grpConsolidatedDf.csv")                

In [105]:
grpDf.head()

,bookingID,absAccX,absAccY,absAccZ,absGyroX,absGyroY,absGyroZ,speedFactor
0,0,0.949421,9.613351,1.719109,0.045798,0.065211,0.041457,0.608393
1,1,0.689900,9.533160,2.210323,0.019148,0.051652,0.022696,0.646962
2,2,0.680615,9.836529,0.817351,0.030824,0.072603,0.021394,0.790034
3,4,0.500273,9.406281,2.614858,0.033129,0.078610,0.045195,0.672072
4,6,0.712031,9.539419,2.371988,0.037086,0.056860,0.035974,0.759399


In [106]:
grpDf.shape

(20000, 8)

In [100]:
labelDf = pd.read_csv('labels.csv')

In [107]:
labelDf.head()

,bookingID,label
0,111669149733,0
1,335007449205,1
2,171798691856,0
3,1520418422900,0
4,798863917116,0


In [101]:
labelDf.shape

(20018, 2)

In [102]:
labelDf.drop_duplicates(subset=['bookingID'],inplace=True,keep='first')

In [103]:
labelDf.shape

(20000, 2)

In [108]:
withLabelDf = pd.merge(labelDf,grpDf,on="bookingID")

In [109]:
withLabelDf.head()

,bookingID,label,absAccX,absAccY,absAccZ,absGyroX,absGyroY,absGyroZ,speedFactor
0,111669149733,0,0.431805,9.772918,0.501404,0.012076,0.038197,0.017437,0.730968
1,335007449205,1,0.808001,9.556147,1.302383,0.042814,0.097799,0.017730,0.696000
2,171798691856,0,0.502351,9.622856,0.381445,0.037971,0.037069,0.044509,0.427330
3,1520418422900,0,0.735054,9.826257,0.840202,0.021369,0.047734,0.024141,0.510179
4,798863917116,0,1.037894,9.617190,1.417894,0.074643,0.077951,0.018139,0.615477


In [111]:
labelDf.to_csv("labels_df.csv")

In [112]:
withLabelDf.to_csv("withLabel_df.csv")

### Preprocssing is done, now lets apply the classification algorithm

In [132]:
allColumns = withLabelDf.columns

In [133]:
excludeLabels = ["bookingID","label"]

In [135]:
featuresCol = [eachCol for eachCol in allColumns if eachCol not in excludeLabels]

In [136]:
featuresCol

['absAccX',
 'absAccY',
 'absAccZ',
 'absGyroX',
 'absGyroY',
 'absGyroZ',
 'speedFactor']

In [160]:
 X_train, X_test, y_train, y_test = train_test_split(withLabelDf[featuresCol],withLabelDf['label'],test_size=0.3, random_state=8912)

In [165]:
y_test

11635    0
19216    0
6751     0
4248     0
9921     0
17523    1
6205     0
17193    0
17783    0
3912     1
13521    0
5338     0
3028     0
17265    0
10740    0
10000    0
7674     0
8373     1
18408    0
12736    0
6633     0
7684     0
10419    0
19721    0
18393    0
14055    0
13602    0
814      0
12119    0
18804    1
        ..
3530     0
6511     1
1561     1
17188    1
12323    0
17664    0
6077     0
17971    0
11632    0
9688     0
18357    1
13332    1
257      1
3457     0
671      0
12705    0
13081    1
13881    0
1102     1
3331     0
12697    0
188      1
15789    0
8561     0
11101    0
4583     0
13341    1
10512    0
8838     0
16637    0
Name: label, Length: 6000, dtype: int64

In [149]:
gnb= GaussianNB()

In [166]:
model = gnb.fit(X_train, y_train)

# Make predictions
preds = gnb.predict(X_test)
print(preds)

# Evaluate accuracy
print(accuracy_score(y_test, preds))

[0 0 0 ... 0 0 0]
0.7441666666666666


In [167]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier().fit(X_train, y_train)
print('Accuracy of Decision Tree classifier on training set: {:.2f}'
     .format(clf.score(X_train, y_train)))
print('Accuracy of Decision Tree classifier on test set: {:.2f}'
     .format(clf.score(X_test, y_test)))

Accuracy of Decision Tree classifier on training set: 1.00
Accuracy of Decision Tree classifier on test set: 0.64


In [168]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
print('Accuracy of K-NN classifier on training set: {:.2f}'
     .format(knn.score(X_train, y_train)))
print('Accuracy of K-NN classifier on test set: {:.2f}'
     .format(knn.score(X_test, y_test)))

Accuracy of K-NN classifier on training set: 0.79
Accuracy of K-NN classifier on test set: 0.72


In [169]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda = LinearDiscriminantAnalysis()
lda.fit(X_train, y_train)
print('Accuracy of LDA classifier on training set: {:.2f}'
     .format(lda.score(X_train, y_train)))
print('Accuracy of LDA classifier on test set: {:.2f}'
     .format(lda.score(X_test, y_test)))

Accuracy of LDA classifier on training set: 0.76
Accuracy of LDA classifier on test set: 0.76


In [170]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(X_train, y_train)
print('Accuracy of GNB classifier on training set: {:.2f}'
     .format(gnb.score(X_train, y_train)))
print('Accuracy of GNB classifier on test set: {:.2f}'
     .format(gnb.score(X_test, y_test)))

Accuracy of GNB classifier on training set: 0.75
Accuracy of GNB classifier on test set: 0.74


In [171]:
from sklearn.svm import SVC
svm = SVC()
svm.fit(X_train, y_train)
print('Accuracy of SVM classifier on training set: {:.2f}'
     .format(svm.score(X_train, y_train)))
print('Accuracy of SVM classifier on test set: {:.2f}'
     .format(svm.score(X_test, y_test)))

C:\Users\pitva01\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


Accuracy of SVM classifier on training set: 0.76
Accuracy of SVM classifier on test set: 0.76


In [172]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
pred = lda.predict(X_test)
print(confusion_matrix(y_test, pred))
print(classification_report(y_test, pred))

[[4449   31]
 [1412  108]]
              precision    recall  f1-score   support

           0       0.76      0.99      0.86      4480
           1       0.78      0.07      0.13      1520

   micro avg       0.76      0.76      0.76      6000
   macro avg       0.77      0.53      0.50      6000
weighted avg       0.76      0.76      0.68      6000



### Lets save the model to file

In [ ]:
filename = 'finalized_model.sav'
pickle.dump(model, open(filename, 'wb'))
 

###  Put the unseen data (against which my model to be evaluated) in unseenData.csv in the same format as that competition file part-00000-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv

In [182]:
    df = pd.read_csv("unseenData.csv")                
                
    df["speedFactor"] = (np.sqrt((df['acceleration_x'])*(df['acceleration_x']) + (df['acceleration_y'])*(df['acceleration_y']) + (df['acceleration_z'])*(df['acceleration_z'])))/(10 + df["Speed"])        
        
    df["absAccX"] = np.abs(df['acceleration_x'])
    df["absAccY"] = np.abs(df['acceleration_y'])
    df["absAccZ"] = np.abs(df['acceleration_z'])

    df["absGyroX"] = np.abs(df['gyro_x'])
    df["absGyroY"] = np.abs(df['gyro_y'])
    df["absGyroZ"] = np.abs(df['gyro_z'])
                                
    newDf=df.groupby('bookingID').agg({'absAccX':'mean', 'absAccY':'mean','absAccZ':'mean','absGyroX':'mean', 'absGyroY':'mean','absGyroZ':'mean','speedFactor':'mean'})
    newDf = newDf.reset_index()
    perFileDf =  perFileDf.append(newDf)

In [189]:
perFileDf.head()


,bookingID,absAccX,absAccY,absAccZ,absGyroX,absGyroY,absGyroZ,speedFactor
0,0,0.917439,9.588628,1.618996,0.036449,0.063380,0.039019,0.616447
1,1,0.590331,9.462895,2.192611,0.018621,0.040373,0.022342,0.621848
2,2,0.587353,9.792962,0.792379,0.031081,0.105638,0.019730,0.771530
3,4,0.522485,9.433036,2.594428,0.037383,0.078418,0.045463,0.669307
4,6,0.628562,9.521509,2.407400,0.038918,0.052217,0.032318,0.770660


In [185]:
allColumns = withLabelDf.columns
excludeLabels = ["bookingID","label"]

In [186]:
featuresCol = [eachCol for eachCol in allColumns if eachCol not in excludeLabels]

In [191]:
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.predict(perFileDf[featuresCol])
print("This is the model prediction")
print(result)

This is the model prediction
[0 0 0 ... 0 0 0]


###  How to read the unseen data and test our model